In [1]:
#imports
import pandas as pd

In [12]:
#Loading Datasets
brands = pd.read_csv("Dataset/brands.csv") 
finance = pd.read_csv("Dataset/finance.csv")
info = pd.read_csv("Dataset/info.csv")
reviews = pd.read_csv("Dataset/reviews.csv")

In [ ]:
#Merging and dropping null values
merged_df = info.merge(finance, on="product_id")
merged_df = merged_df.merge(reviews, on="product_id")
merged_df = merged_df.merge(brands, on="product_id")
merged_df.dropna(inplace=True)

In [14]:
#Add price labels based on listing_prices quartiles
merged_df["price_label"] = pd.qcut(merged_df["listing_price"],q=4,labels=["Budget", "Average", "Expensive", "Elite"])

In [16]:
#grouping by brand and price label to get volume and mean revenue
adidas_vs_nike = merged_df.groupby(["brand","price_label"], as_index= False).agg(num_products = ("price_label","count"),mean_revenue=("revenue","mean")).round(2)
print(adidas_vs_nike)

    brand price_label  num_products  mean_revenue
0  Adidas      Budget           574       2015.68
1  Adidas     Average           655       3035.30
2  Adidas   Expensive           759       4621.56
3  Adidas       Elite           587       8302.78
4    Nike      Budget           357       1596.33
5    Nike     Average             8        675.59
6    Nike   Expensive            47        500.56
7    Nike       Elite           130       1367.45


C:\Users\Pichau\AppData\Local\Temp\ipykernel_15932\2056845277.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adidas_vs_nike = merged_df.groupby(["brand","price_label"], as_index= False).agg(num_products = ("price_label","count"),mean_revenue=("revenue","mean")).round(2)


In [17]:
#Store the length of each description
merged_df["description_length"] = merged_df["description"].str.len()

In [20]:
#Cut into bins
merged_df["description_length"] = pd.cut(merged_df["description_length"],bins = [0, 100, 200, 300, 400, 500, 600, 700], labels =["100", "200", "300", "400", "500", "600", "700"])
#Groupby the bins
description_lengths = merged_df.groupby("description_length", as_index=False).agg(mean_rating = ("rating","mean"),num_reviews=("reviews","count"))
print(description_lengths)

  description_length  mean_rating  num_reviews
0                100     2.257143            7
1                200     3.185551          526
2                300     3.277815         1785
3                400     3.285407          651
4                500     3.345763          118
5                600     3.120000           15
6                700     3.653333           15


C:\Users\Pichau\AppData\Local\Temp\ipykernel_15932\4273540362.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  description_lengths = merged_df.groupby("description_length", as_index=False).agg(mean_rating = ("rating","mean"),num_reviews=("reviews","count"))


In [21]:
#Now we search for variation in volume of products and mediana revenue between clothing and footwear
# List of footwear keywords
mylist = "shoe*|trainer*|foot*"
# Filter for footwear products
shoes = merged_df[merged_df["description"].str.contains(mylist)]
# Filter for clothing products
clothing = merged_df[~merged_df.isin(shoes["product_id"])]
# Remove null product_id values from clothing DataFrame
clothing.dropna(inplace=True)
# Create product_types DataFrame
product_types = pd.DataFrame({"num_clothing_products": len(clothing), 
                              "median_clothing_revenue": clothing["revenue"].median(), 
                              "num_footwear_products": len(shoes), 
                              "median_footwear_revenue": shoes["revenue"].median()}, 
                              index=[0])

print(product_types)

   num_clothing_products  median_clothing_revenue  num_footwear_products  \
0                    478                   625.07                   2639   

   median_footwear_revenue  
0                   3073.3  
